<a href="https://colab.research.google.com/github/stefanogiagu/corso_AI_2023/blob/main/esonero2/MetodiAI_Fis2023_Esonero_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Seconda Prova Esonero - Metodi AI e Machine Learning per la fisica
### 15.6.2023 - AA 2022/23 - Docente: S. Giagu


> **Regole:**

*   **tempo a disposizione:** 2h' + 20' per scaricare il dataset / fare l'upload del notebook a fine esonero

*   compilare con i vostri dati i campi della cella che segue e poi eseguire la cella verificando che i dati printati corrispondano. L'esecuzione della cella scarica anche il dataset necessartio (~10' di tempo in media)
*   risolvere i quesiti/compiti indicati nella cella *Descrizione del compito* usando questo notebook
*   una volta completato il compito sottomettere il notebook nel apposito folder sul sito e-learning del corso disponibile al link: <p>
[consegna esonero](https://elearning.uniroma1.it/mod/assign/view.php?id=567302)<p>
NOTA BENE: una volta sottomesso il notebook non sono più possibili ulteriori modifiche

In [ ]:
#@ Dati Personali
import os

Nome = 'Stefano'  #@param {type: "string"}
Cognome = 'Giagu' #@param {type: "string"}
NumeroMatricola = 12345678 #@param {type: "number"}

if NumeroMatricola == 12345678:
  print('\033[1;31m Inserisci il numero di matricola corretto!!!!')
else:
  print('Download dataset ...')
  !wget http://giagu.web.cern.ch/giagu/CERN/dataset_c.zip
  !unzip -q dataset_c.zip
  print('Done')

# Descrizione del compito:

Viene fornito un dataset costituto da:

*   50k imagini jpeg di quadranti di orologi analogici (shape: 1x100x100)
*   un file numpy compresso (*.npz*) che contiene due vettory numpy che forniscono per ciascuna immagine il valore dell'ora e dei minuti corrispondente

Viene fornito il codice per costruire un Dataset pytorch fromato per ogni campione da un'immagine, una label discreta corrispondete al valore dell'ora mostrata dall'orologio (0,1,2,3...,11), e un target continuo corrispondente al valore dei minuti.



**È richiesto di risolvere i seguenti task:**

1.   graficare qualche esempio di immagine del dataset e l'orario corrispondente (ore:minuti)
2.   suddividere il pytorch dataset in quattro sub-datastes di training, validation, test e resto, costituti rispettivamente da 10k eventi, 1k eventi, 1k eventi, e 38k eventi (NOTA: usare eventualmente più di 10k eventi per migliorare le prestazioni fa si che il training si allunghi notevolmente, quindi pensateci bene prima di seguire questa strada)
3.   allenare un modello convoluzionale in modo che predica (task regressione) il target continuo del valore dei minuti associato ad ogni immagine, e stimarne opporunamente le prestazioni
4.   prendere 4 esempi random dal test set e predirne i minuti, graficando l'immagine corrispondente con titolo i minuti predetti.
5.   provare a modificare il modello in modo che funzioni in modalità multi-task: classificazione + regressione, producendo in output la predizione della classe oraria, e il valore continuo del target dei minuti. Si richiede di: scrivere il modello pytorch + definire le loss/metriche opportunem scrivere il training loop necessario per allenare il modello. Non è richiesto invece l'allenamento del modello stesso che richiede tempi più lunghi del modello di sola classificazione.

* commentare opportunamente i risultati ottenuti in ogni punto

In [ ]:
# Lettura dei dati e costruzione del pytorch dataset

import os.path
from torchsummary import summary

import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models


# legge label e target
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

filet = np.load('dataset_c/targets.npz')

hlabel = filet['hlabel']
mvalue = filet['mvalue']

hlabel = torch.Tensor(hlabel).int()
mvalue = torch.Tensor(mvalue).float()

print(hlabel.shape)
print(mvalue.shape)


# crea il custom dataset
from torch.utils.data import Dataset, DataLoader
import os
import natsort

class CustomDataSet(Dataset):
    def __init__(self, main_dir, transform):
        self.main_dir = main_dir
        self.transform = transform
        all_imgs = os.listdir(main_dir)
        self.total_imgs = natsort.natsorted(all_imgs)

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = Image.open(img_loc).convert('L')
        tensor_image = self.transform(image)
        y1 = hlabel[idx]
        y2 = mvalue[idx]
        return tensor_image, y1, y2 #ogni evento è formato da il tensore pytroch dell'immagine, un tensore intero della label orario, e un tensore continuo del target dei minuti


train_transforms = transforms.Compose([
                                       transforms.Resize((100,100)), #riscala la dimensione delle immagini da 300x300 a 100x100 per ridurre i tempi di training
                                       transforms.ToTensor(), #converte in tensori torch
                                      ])



# custom pytorch dataset
dataset = CustomDataSet('dataset_c/images/', transform=train_transforms)

print('Dataset size: ', len(dataset))

In [ ]:
# ... codice